### ConversationKGMemory

지식 그래프의 힘을 활용하여 정보를저장하고 불러온다.

이를 통해 모델이 서로 다른 개체 간의 관계를 이해하는데 도움을 주고, 복잡한 연결망과 역사적 맥락을 기반으로 대응하는 능력을 향상시킨다

- KG 는 Knowledge Graph의 약자다. 지식그래프.
- 핵심정보를 지식그래프 기반이냐, Entity를 뽑아서 저장할것이냐를 정하는 것이다.

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationKGMemory

In [4]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [8]:
memory = ConversationKGMemory(llm=llm, return_messages=True)
memory.save_context(
    {"input": "이쪽은 Dongtan 에 거주중인 김재호씨 입니다."},
    {"output": "김재호씨는 누구시죠?"},
)
memory.save_context(
    {"input": "김재호씨는 우리 회사의 신입 RAG 개발자입니다."},
    {"output": "만나서 반갑습니다."},
)

In [9]:
memory.load_memory_variables({"input": "김재호씨는 누구입니까?"})

{'history': [SystemMessage(content='On 김재호씨: 김재호씨 is a 신입 RAG 개발자. 김재호씨 is in 우리 회사.')]}

### Chain의 메모리에 활용

In [10]:
from langchain_core.prompts import PromptTemplate
from langchain.chains import ConversationChain

In [14]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

template = """The following is a friendly conversation between a human and an AI. 
The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know. 
The AI ONLY uses information contained in the "Relevant Information" section and does not hallucinate.

Relevant Information:

{history}

Conversation:
Human: {input}
AI:"""

prompt = PromptTemplate.from_template(
    template,
    #input_variables=['history', 'input']
)

conversation_with_kg = ConversationChain(
    llm=llm, prompt=prompt, memory=ConversationKGMemory(llm=llm)
)

In [15]:
conversation_with_kg.predict(
    input="My name is Teddy. Shirley is a coworker of mine, and she's a new designer at our company."
)

"Hi Teddy! It's great to meet you. How exciting that Shirley is a new designer at your company! What kind of design work does she do? Is she focusing on graphic design, product design, or something else?"

In [16]:
# Shirley 에 대한 질문
conversation_with_kg.memory.load_memory_variables({"input": "who is Shirley?"})

{'history': 'On Shirley: Shirley is a coworker. Shirley is a new designer. Shirley is at our company.'}